
<body>
<h1>ISDA 111 - Video Game Data Analytics Project</h1>
<p>Hi team, This is the starter document we can use.</p>
<h2>Team Members: add your name here</h2>
<p>Chris Alcala - Handimexi</p>
<p>add here</p>
<p>add here</p>

</body>

